# Capstone Project - The Battle of Neighborhoods

## Business Problem Section

#### Background

London is the largest city and the capital of United Kingdom. It is considered as one of the most important global cities. Its estimated mid-2018 municipal population (corresponding to Greater London) was 8,908,081, the third most populous of any city in Europe and accounts for 13.4% of the UK population. The city covers a total area of 607 sq. miles. Clearly, London is a city with high population density. Real estate investment in such cities thus becomes a big investment. And before making such an investment, one should have a proper investment strategy.

#### Business Problem

In a country with such a huge area and high real estate prices, it becomes very tough to find a property with its appropriate value, keeping in mind all the amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores. As a result, the business problem we are currently posing is: How could we recommend profitable venues to support homebuyers according to amenities and essential facilities surrounding such venues?

## Data Section

Data on London properties and the relative price paid data were extracted from the HM Land Registry (http://landregistry.data.gov.uk/).
The following fields comprise the address data included in Price Paid Data: Postcode; PAON Primary Addressable Object Name, typically the house number or name; SAON Secondary Addressable Object Name. If there is a sub-building, for example, the building is divided into flats, there will be a SAON; Street; Locality; Town/City; District; County.

To explore and target recommended locations across different venues according to the presence of amenities and essential facilities, we will access data through FourSquare API interface and arrange them as a dataframe for visualization.

By merging data on London properties and the relative price paid data from the HM Land Registry and data on amenities and essential facilities surrounding such properties from FourSquare API interface, we will be able to recommend profitable real estate investments.

## Methodology

This section will describe the main componeants of our anaysis and prediction system. It consisits of the following sections:
1. Data Collection
2. Exploring and understanding the data.
3. Data preparation and processing
4. Data Modeling

#### 1. Data Collection

We'll start with first importing the required libraries.

In [2]:
import numpy as np
import pandas as pd
import datetime as dt # Datetime
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge folium=0.5.0 --yes
import folium #import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0          conda-forge
    geopy:           

Now we will downlod data from the HM Land Registry

In [3]:
df_ppd = pd.read_csv("http://prod2.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-2018.csv")

#### 2. Exploring and understanding the data.

Before using the data, it is important to understand it.
Let's check the first five rows to see what our data contains.

In [4]:
df_ppd.head()

,{79A74E21-D11E-1289-E053-6B04A8C01627},770000,2018-09-25 00:00,SK7 1AR,D,N,F,5,Unnamed: 8,OAK MEADOW,BRAMHALL,STOCKPORT,STOCKPORT.1,GREATER MANCHESTER,A,A.1
0,{79A74E21-D11F-1289-E053-6B04A8C01627},253500,2018-09-24 00:00,M6 8GQ,D,N,F,1,NaN,RIVINGTON ROAD,NaN,SALFORD,SALFORD,GREATER MANCHESTER,A,A
1,{79A74E21-D120-1289-E053-6B04A8C01627},231950,2018-09-28 00:00,WA3 2UE,D,Y,F,35,NaN,STONEACRE CLOSE,LOWTON,WARRINGTON,WIGAN,GREATER MANCHESTER,A,A
2,{79A74E21-D121-1289-E053-6B04A8C01627},112500,2018-08-29 00:00,OL6 6RJ,S,N,F,102,NaN,THORNFIELD GROVE,NaN,ASHTON-UNDER-LYNE,TAMESIDE,GREATER MANCHESTER,A,A
3,{79A74E21-D122-1289-E053-6B04A8C01627},184995,2018-06-15 00:00,M46 0TW,S,Y,F,37,NaN,THREADNEEDLE PLACE,ATHERTON,MANCHESTER,WIGAN,GREATER MANCHESTER,A,A
4,{79A74E21-D123-1289-E053-6B04A8C01627},214995,2018-09-28 00:00,M28 3XS,D,Y,L,9,NaN,MARPLE GARDENS,WORSLEY,MANCHESTER,SALFORD,GREATER MANCHESTER,A,A


In [5]:
df_ppd.shape

(1029929, 16)

Our data has 16 columns and more than 1,000,000 rows. We shall now prepare and preprocess data acoording to our requirement.

#### 3. Preparing and Processing the data

At this stage we convert our data from intial raw form into another preprocessed form inorder to prepare the data for further analysis. Therefore we perform the following steps:
- Rename the columns
- Format the date column and sort the data by date of sale
- Make a list of street names in London
- Calculate the street-wise average price of the property
- Read the street-wise coordinates into a data frame, eliminating re-occuring word London from the indivisual names
- Join the data to find the coordinates that fits into our client's budget
- Plot recommended locations on London map along with cureent market prices

Assign meaningful names to the columns

In [64]:
df_ppd.columns=['TUID', 'Price', 'Date_Transfer', 'Postcode', 'Prop_Type', 'Old_New', 'Duration', 'PAON', \
                  'SAON', 'Street', 'Locality', 'Town_City', 'District', 'County', 'PPD_Cat_Type', 'Record_Status']

Next step is to format the date column

In [7]:
df_ppd['Date_Transfer']=df_ppd['Date_Transfer'].apply(pd.to_datetime)

In [8]:
# Delete all obsolete transactions which were done before 2018
df_ppd.drop(df_ppd[df_ppd.Date_Transfer.dt.year < 2018].index, inplace=True)

# Sort by Date of Sale
df_ppd.sort_values(by=['Date_Transfer'],ascending=[False],inplace=True)

Now, lets list down the streets in London

In [9]:
df_ppd_london = df_ppd.query("Town_City == 'LONDON'")
streets = df_ppd_london['Street'].unique().tolist()

In [10]:
streets

['BRIDGE LANE',
 'HERMITAGE ROAD',
 'CAMLEY STREET',
 'ST JOSEPHS STREET',
 'SHEPHERDESS WALK',
 'DERWENT ROAD',
 'VICTORIA STREET',
 'BROSTER GARDENS',
 'SYLVAN HILL',
 'THREE OAK LANE',
 'GLASSHOUSE GARDENS',
 'QUICKS ROAD',
 'ST GEORGES SQUARE',
 'HYDE ESTATE ROAD',
 'RITHERDON ROAD',
 'TRINITY CRESCENT',
 'APPLE YARD',
 'LANCASTER GATE',
 'HANDYSIDE STREET',
 'MICHLEHAM DOWN',
 'CHINGFORD MOUNT ROAD',
 'GRAND DRIVE',
 'EXCHANGE GARDENS',
 'BRUSHFIELD STREET',
 'WENTWORTH STREET',
 'BUCKINGHAM PALACE ROAD',
 'CLIFFORD STREET',
 'HOWARD ROAD',
 'BLACKFRIARS ROAD',
 'HIGHWOOD HILL',
 "ST JAMES'S PLACE",
 'FLORAL STREET',
 'RYMILL STREET',
 'COLLENDALE ROAD',
 'TURNHAM GREEN TERRACE',
 'PALACE COURT',
 'HONOUR LEA AVENUE',
 'ARCHWAY ROAD',
 'HARTFIELD ROAD',
 'GALLOWAY ROAD',
 'WARWICK WAY',
 'OXFORD STREET',
 'UPPER TERRACE',
 'WOOLF CLOSE',
 'FULHAM HIGH STREET',
 'ABBOTS MANOR',
 'LOWER BOSTON ROAD',
 'PRINCE ALBERT ROAD',
 'UPPER RICHMOND ROAD',
 'NORTH BIRKBECK ROAD',
 'CLIFFORD R

We now calculate the street-wise average price of the property.

In [11]:
df_grp_price = df_ppd_london.groupby(['Street'])['Price'].mean().reset_index()

# Giving column names
df_grp_price.columns = ['Street', 'Avg_Price']

In [12]:
#Input your Budget's Upper Limit and Lower Limit - Find the locations df_grp_price which fits your budget
df_affordable = df_grp_price.query("(Avg_Price >= 2200000) & (Avg_Price <= 2500000)")
df_affordable

,Street,Avg_Price
196,ALBION SQUARE,2.450000e+06
390,ANHALT ROAD,2.435000e+06
405,ANSDELL TERRACE,2.250000e+06
422,APPLEGARTH ROAD,2.400000e+06
855,BARONSMEAD ROAD,2.375000e+06
981,BEAUCLERC ROAD,2.480000e+06
1102,BELVEDERE DRIVE,2.340000e+06
1215,BICKENHALL STREET,2.208500e+06
1253,BIRCHLANDS AVENUE,2.217000e+06
1553,BRAMPTON GROVE,2.456875e+06


We have 162 street locations that fits into our clint's budget

In [13]:
import pandas as pd
import numpy as np
import datetime as DT
import hmac
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# import k-means from clustering stage
from sklearn.cluster import KMeans

In [14]:
geolocator = Nominatim(user_agent="ldn_explorer")
df_affordable['city_coord'] = df_affordable['Street'].apply(geolocator.geocode).apply(lambda x: (x.latitude, x.longitude))
df_affordable[['Latitude', 'Longitude']] = df_affordable['city_coord'].apply(pd.Series)
df = df_affordable.drop(columns=['city_coord'])
df

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


,Street,Avg_Price,Latitude,Longitude
196,ALBION SQUARE,2.450000e+06,-41.273758,173.289393
390,ANHALT ROAD,2.435000e+06,51.480316,-0.166801
405,ANSDELL TERRACE,2.250000e+06,51.499890,-0.189103
422,APPLEGARTH ROAD,2.400000e+06,53.748654,-0.326670
855,BARONSMEAD ROAD,2.375000e+06,51.477315,-0.239457
981,BEAUCLERC ROAD,2.480000e+06,30.211452,-81.617981
1102,BELVEDERE DRIVE,2.340000e+06,44.707562,-63.545599
1215,BICKENHALL STREET,2.208500e+06,51.521201,-0.158908
1253,BIRCHLANDS AVENUE,2.217000e+06,51.448394,-0.160468
1553,BRAMPTON GROVE,2.456875e+06,51.589961,-0.318525


In [15]:

address = 'London, UK'

geolocator = Nominatim(user_agent="ldn_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of London City are 51.5073219, -0.1276474.


In [16]:
map_london = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, price, street in zip(df['Latitude'], df['Longitude'], df['Avg_Price'], df['Street']):
    label = '{}, {}'.format(street, price)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
map_london

In [17]:
#Define Foursquare Credentials and Version

CLIENT_ID = 'AG4W05YY1I5QC5TFIGI3AXDTDMTAEDEELHNW2SFQJAA5EELF' # your Foursquare ID
CLIENT_SECRET = 'CO1HYE001KSTDVZ2TG5GSLN0M2YGWATVUBQSCHGGNU5XL1R4' 
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: AG4W05YY1I5QC5TFIGI3AXDTDMTAEDEELHNW2SFQJAA5EELF
CLIENT_SECRET:CO1HYE001KSTDVZ2TG5GSLN0M2YGWATVUBQSCHGGNU5XL1R4


We have located the affordable coordinated into the map. We can now proceed to the Modeling phase. We will analyze the neighborhoods to recommend real estates where hom buyers can make their investments. We will then recommend them profitable venues according to amenities and essential facilities in the surrounding area.

#### 4. Modeling

We will use the K-Means technique as it is the fastest and efficient in terms of computational cost, is highly flexible to account for mutations in real estate market in London.

In [18]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Street', 
                  'Street Latitude', 
                  'Street Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [19]:
# Run the above function on each location and create a new dataframe called location_venues and display it.
location_venues = getNearbyVenues(names=df['Street'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

ALBION SQUARE
ANHALT ROAD
ANSDELL TERRACE
APPLEGARTH ROAD
BARONSMEAD ROAD
BEAUCLERC ROAD
BELVEDERE DRIVE
BICKENHALL STREET
BIRCHLANDS AVENUE
BRAMPTON GROVE
BRIARDALE GARDENS
BROOKWAY
BURBAGE ROAD
BURY WALK
CALLCOTT STREET
CAMPDEN HILL ROAD
CAMPION ROAD
CANNING PLACE
CARLISLE ROAD
CARLTON GARDENS
CARLYLE COURT
CHALCOT SQUARE
CHARLES LANE
CHELSEA CRESCENT
CHESTER CLOSE NORTH
CHEYNE COURT
CHEYNE ROW
CHISWICK MALL
CITY ROAD
CLARENDON STREET
CLONCURRY STREET
COLBECK MEWS
COLLEGE CRESCENT
CORNWALL TERRACE MEWS
COURT LANE GARDENS
CRESCENT GROVE
DALEBURY ROAD
DEWHURST ROAD
DORIA ROAD
DOWNSHIRE HILL
DUCHESS WALK
ECCLESTON SQUARE MEWS
EGBERT STREET
EGERTON PLACE
ELM PARK ROAD
FLORAL STREET
FRANK DIXON WAY
FULTON MEWS
GERARD ROAD
GERRARD ROAD
GIRDLERS ROAD
GLOUCESTER CRESCENT
GORDON PLACE
GRAFTON SQUARE
GRAHAM TERRACE
HARMAN DRIVE
HARRIS STREET
HAVANNAH STREET
HAZLEWELL ROAD
HEREFORD MEWS
HERONDALE AVENUE
HIGHGATE HIGH STREET
HIGHWOOD HILL
HILLGATE PLACE
HOLLYCROFT AVENUE
HOLLYWOOD MEWS
HONEYWELL

In [20]:
location_venues

,Street,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,ALBION SQUARE,-41.273758,173.289393,The Free House,-41.273340,173.287364,Bar
1,ALBION SQUARE,-41.273758,173.289393,The Indian Cafe,-41.273308,173.286530,Indian Restaurant
2,ALBION SQUARE,-41.273758,173.289393,Queen's Gardens,-41.273671,173.291383,Park
3,ALBION SQUARE,-41.273758,173.289393,Urban,-41.274355,173.286317,New American Restaurant
4,ALBION SQUARE,-41.273758,173.289393,Fish Stop,-41.276010,173.289592,Fish & Chips Shop
5,ALBION SQUARE,-41.273758,173.289393,Deville Cafe,-41.271941,173.285535,Beer Garden
6,ALBION SQUARE,-41.273758,173.289393,The Bridge Street Collective,-41.272520,173.285517,Café
7,ALBION SQUARE,-41.273758,173.289393,Mango,-41.274460,173.285345,Indian Restaurant
8,ALBION SQUARE,-41.273758,173.289393,Hopgood's,-41.274749,173.283831,Restaurant
9,ALBION SQUARE,-41.273758,173.289393,Sprig & Fern,-41.274508,173.286527,Brewery


In [21]:
location_venues.groupby('Street').count()

,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Street,,,,,,
ALBION SQUARE,26,26,26,26,26,26
ANHALT ROAD,15,15,15,15,15,15
ANSDELL TERRACE,45,45,45,45,45,45
APPLEGARTH ROAD,4,4,4,4,4,4
BARONSMEAD ROAD,14,14,14,14,14,14
BEAUCLERC ROAD,4,4,4,4,4,4
BELVEDERE DRIVE,1,1,1,1,1,1
BICKENHALL STREET,61,61,61,61,61,61
BIRCHLANDS AVENUE,11,11,11,11,11,11


In [22]:
# get the List of Unique Categories
print('There are {} uniques categories.'.format(len(location_venues['Venue Category'].unique())))

There are 348 uniques categories.


In [23]:
# one hot encoding
venues_onehot = pd.get_dummies(location_venues[['Venue Category']], prefix="", prefix_sep="")

# add street column back to dataframe
venues_onehot['Street'] = location_venues['Street'] 

# move street column to the first column
fixed_columns = [venues_onehot.columns[-1]] + list(venues_onehot.columns[:-1])

#fixed_columns
venues_onehot = venues_onehot[fixed_columns]

venues_onehot.head()

,Street,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,...,Vietnamese Restaurant,Warehouse Store,Waterfront,Weight Loss Center,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
london_grouped = venues_onehot.groupby('Street').mean().reset_index()
london_grouped

,Street,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,...,Vietnamese Restaurant,Warehouse Store,Waterfront,Weight Loss Center,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,ALBION SQUARE,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
1,ANHALT ROAD,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
2,ANSDELL TERRACE,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
3,APPLEGARTH ROAD,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
4,BARONSMEAD ROAD,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
5,BEAUCLERC ROAD,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
6,BELVEDERE DRIVE,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
7,BICKENHALL STREET,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.016393,0.016393,0.0,0.0
8,BIRCHLANDS AVENUE,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
9,BRAMPTON GROVE,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0


In [25]:
london_grouped.shape

(149, 349)

In [26]:
# Define a function to return the most common venues/facilities nearby real estate investments#
num_top_venues = 10

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [27]:
indicators = ['st', 'nd', 'rd']
num_top_venues = 10
# creaye columns accprding to number of top venues
columns = ['Street']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

In [28]:
# create a new dataframe
venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['Street'] = london_grouped['Street']

for ind in np.arange(london_grouped.shape[0]):
    venues_sorted.iloc[ind, 1:] = return_most_common_venues(london_grouped.iloc[ind, :], num_top_venues)

In [29]:
venues_sorted.head()

,Street,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ALBION SQUARE,Café,Indian Restaurant,Pub,Coffee Shop,Restaurant,Bar,Burger Joint,New American Restaurant,Seafood Restaurant,Museum
1,ANHALT ROAD,Pub,French Restaurant,Grocery Store,Diner,Plaza,Japanese Restaurant,Gym / Fitness Center,Cocktail Bar,English Restaurant,Garden
2,ANSDELL TERRACE,Juice Bar,Hotel,Pub,Restaurant,Indian Restaurant,Clothing Store,Italian Restaurant,Café,Supermarket,Mediterranean Restaurant
3,APPLEGARTH ROAD,Pub,Nightclub,Bar,Casino,Food & Drink Shop,Food,Food Service,Exhibit,Factory,Falafel Restaurant
4,BARONSMEAD ROAD,Pub,Thai Restaurant,Pizza Place,Community Center,Restaurant,Coffee Shop,Park,Farmers Market,Café,Nature Preserve


In [30]:
venues_sorted.shape

(149, 11)

In [31]:
london_grouped.shape

(149, 349)

In [32]:
london_grouped=df

After our inspection of venues, facilities and amenities nearby the most profitable real estate in Loondon, we cab now start clustering properties accordingly.

We will be distributing the area into 10 clusters.

In [46]:
# set number of clusters
kclusters = 10

london_grouped_clustering = london_grouped.drop('Street', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(london_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:50]

array([7, 1, 8, 9, 3, 4, 5, 2, 2, 7, 9, 9, 7, 4, 3, 3, 7, 1, 2, 4, 6, 0,
       9, 4, 7, 8, 9, 0, 7, 8, 9, 3, 1, 5, 3, 6, 9, 1, 3, 2, 4, 5, 0, 2,
       6, 2, 2, 6, 8, 8], dtype=int32)

In [47]:
#Dataframe to include Clusters

london_grouped_clustering=df
london_grouped_clustering.head()

,Street,Avg_Price,Latitude,Longitude,Cluster Labels
196,ALBION SQUARE,2450000.0,-41.273758,173.289393,1
390,ANHALT ROAD,2435000.0,51.480316,-0.166801,3
405,ANSDELL TERRACE,2250000.0,51.499890,-0.189103,0
422,APPLEGARTH ROAD,2400000.0,53.748654,-0.326670,3
855,BARONSMEAD ROAD,2375000.0,51.477315,-0.239457,2


In [48]:
london_grouped_clustering.shape

(162, 5)

In [49]:
london_grouped_clustering.dtypes

Street             object
Avg_Price         float64
Latitude          float64
Longitude         float64
Cluster Labels      int32
dtype: object

In [50]:
df.shape

(162, 5)

In [51]:
df.dtypes

Street             object
Avg_Price         float64
Latitude          float64
Longitude         float64
Cluster Labels      int32
dtype: object

In [52]:

# add clustering labels
london_grouped_clustering['Cluster Labels'] = kmeans.labels_

# merge london_grouped with london_data to add latitude/longitude for each neighborhood
london_grouped_clustering = london_grouped_clustering.join(venues_sorted.set_index('Street'), on='Street')

london_grouped_clustering.head(30) # check the last columns!

,Street,Avg_Price,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
196,ALBION SQUARE,2.450000e+06,-41.273758,173.289393,7,Café,Indian Restaurant,Pub,Coffee Shop,Restaurant,Bar,Burger Joint,New American Restaurant,Seafood Restaurant,Museum
390,ANHALT ROAD,2.435000e+06,51.480316,-0.166801,1,Pub,French Restaurant,Grocery Store,Diner,Plaza,Japanese Restaurant,Gym / Fitness Center,Cocktail Bar,English Restaurant,Garden
405,ANSDELL TERRACE,2.250000e+06,51.499890,-0.189103,8,Juice Bar,Hotel,Pub,Restaurant,Indian Restaurant,Clothing Store,Italian Restaurant,Café,Supermarket,Mediterranean Restaurant
422,APPLEGARTH ROAD,2.400000e+06,53.748654,-0.326670,9,Pub,Nightclub,Bar,Casino,Food & Drink Shop,Food,Food Service,Exhibit,Factory,Falafel Restaurant
855,BARONSMEAD ROAD,2.375000e+06,51.477315,-0.239457,3,Pub,Thai Restaurant,Pizza Place,Community Center,Restaurant,Coffee Shop,Park,Farmers Market,Café,Nature Preserve
981,BEAUCLERC ROAD,2.480000e+06,30.211452,-81.617981,4,Spa,Automotive Shop,Harbor / Marina,Pizza Place,Zoo Exhibit,Filipino Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant
1102,BELVEDERE DRIVE,2.340000e+06,44.707562,-63.545599,5,Campground,Zoo Exhibit,Film Studio,Exhibit,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant
1215,BICKENHALL STREET,2.208500e+06,51.521201,-0.158908,2,Gastropub,Restaurant,Coffee Shop,Pizza Place,Hotel,Italian Restaurant,Garden,Bakery,Greek Restaurant,Bar
1253,BIRCHLANDS AVENUE,2.217000e+06,51.448394,-0.160468,2,Pub,Breakfast Spot,Coffee Shop,Brewery,French Restaurant,Chinese Restaurant,Lake,Train Station,Bakery,Pizza Place
1553,BRAMPTON GROVE,2.456875e+06,51.589961,-0.318525,7,Food Service,Home Service,Zoo Exhibit,Film Studio,Event Space,Exhibit,Factory,Falafel Restaurant,Farm,Farmers Market


In [53]:

# Create Map

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(london_grouped_clustering['Latitude'], london_grouped_clustering['Longitude'], london_grouped_clustering['Street'], london_grouped_clustering['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [54]:

london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 0, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2406,2.286679e+06,Café,Pub,Bar,Coffee Shop,Italian Restaurant,French Restaurant,Park,Convenience Store,Spa,Speakeasy
2686,2.287500e+06,Pub,Art Museum,Gift Shop,Gym / Fitness Center,Brewery,Cupcake Shop,Event Space,Factory,Coworking Space,Falafel Restaurant
4285,2.265000e+06,Gym / Fitness Center,American Restaurant,Trail,Gym,Fast Food Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Factory
5486,2.270000e+06,Pub,Café,Burger Joint,Restaurant,Bar,Cocktail Bar,Coffee Shop,Gym / Fitness Center,Grocery Store,Italian Restaurant
5952,2.262500e+06,River,Zoo Exhibit,Filipino Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant,Farm,Farmers Market


In [55]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 1, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
390,2.435000e+06,Pub,French Restaurant,Grocery Store,Diner,Plaza,Japanese Restaurant,Gym / Fitness Center,Cocktail Bar,English Restaurant,Garden
2158,2.425000e+06,Hotel,Italian Restaurant,Juice Bar,Indian Restaurant,Chinese Restaurant,Mediterranean Restaurant,French Restaurant,Restaurant,Clothing Store,Pub
2995,2.440000e+06,Indian Restaurant,Department Store,Shopping Mall,Shipping Store,Paper / Office Supplies Store,Liquor Store,Laundromat,Pharmacy,Coffee Shop,Resort
3848,2.425000e+06,Pub,Weight Loss Center,Grocery Store,Food & Drink Shop,Food,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Factory
5289,2.441667e+06,Pub,Hotel,Convention Center,Park,Sandwich Place,Gastropub,Italian Restaurant,Indian Restaurant,Bike Rental / Bike Share,Bar


In [56]:

london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 2, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1215,2208500.0,Gastropub,Restaurant,Coffee Shop,Pizza Place,Hotel,Italian Restaurant,Garden,Bakery,Greek Restaurant,Bar
1253,2217000.0,Pub,Breakfast Spot,Coffee Shop,Brewery,French Restaurant,Chinese Restaurant,Lake,Train Station,Bakery,Pizza Place
2225,2200000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3980,2225000.0,Café,Pub,Italian Restaurant,Bakery,Coffee Shop,Museum,Thai Restaurant,Bookstore,Bistro,Grocery Store
4289,2200000.0,Indian Restaurant,Grocery Store,Sandwich Place,Café,Thai Restaurant,Bar,Pub,Home Service,Eastern European Restaurant,Electronics Store


In [57]:

london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 3, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
855,2375000.0,Pub,Thai Restaurant,Pizza Place,Community Center,Restaurant,Coffee Shop,Park,Farmers Market,Café,Nature Preserve
2068,2375000.0,Pub,Park,Ice Cream Shop,Bakery,Indian Restaurant,Hotel,Yoga Studio,Burger Joint,Caribbean Restaurant,Sandwich Place
2129,2379652.7,Pub,Bakery,Indian Restaurant,Ice Cream Shop,Coffee Shop,Park,Hotel,Hostel,Yoga Studio,Breakfast Spot
2944,2367500.0,Hotel,Pub,Garden,Café,Coffee Shop,Italian Restaurant,Chinese Restaurant,Bar,Mediterranean Restaurant,Restaurant
3255,2360000.0,Pub,Grocery Store,Pizza Place,Café,Furniture / Home Store,Greek Restaurant,Art Gallery,Italian Restaurant,Bakery,Wine Shop


In [58]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 4, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
981,2480000.0,Spa,Automotive Shop,Harbor / Marina,Pizza Place,Zoo Exhibit,Filipino Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant
1980,2492500.0,Supermarket,English Restaurant,Rental Car Location,Gym,Hardware Store,Fast Food Restaurant,Coffee Shop,Café,Dry Cleaner,Park
2230,2483500.0,Italian Restaurant,Dessert Shop,Lebanese Restaurant,Convenience Store,Japanese Restaurant,Coffee Shop,Light Rail Station,Ice Cream Shop,Deli / Bodega,Hotel
2562,2495000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4035,2477500.0,Coffee Shop,Pub,Bar,Cocktail Bar,Italian Restaurant,Thai Restaurant,Hotel,French Restaurant,Museum,English Restaurant


In [59]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 5, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1102,2.340000e+06,Campground,Zoo Exhibit,Film Studio,Exhibit,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant
3202,2.350000e+06,Café,Museum,Coffee Shop,Garden,Park,Thai Restaurant,Pizza Place,Tennis Court,Movie Theater,Lake
4232,2.335500e+06,Hotel,Pub,Sandwich Place,Café,Italian Restaurant,Coffee Shop,Bakery,Restaurant,Gym / Fitness Center,Turkish Restaurant
5382,2.350833e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5493,2.325000e+06,Indian Restaurant,Café,Film Studio,Exhibit,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant


In [60]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 6, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2242,2300000.0,Farm,Soup Place,Zoo Exhibit,Film Studio,Event Space,Exhibit,Factory,Falafel Restaurant,Farmers Market,Fast Food Restaurant
3377,2298000.0,Hotel,Zoo Exhibit,Film Studio,Event Space,Exhibit,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
4375,2320425.0,Café,Supermarket,Platform,Park,Gym / Fitness Center,Stationery Store,Bus Station,Furniture / Home Store,Fast Food Restaurant,Breakfast Spot
5098,2299000.0,Hotel,Pub,Coffee Shop,Chinese Restaurant,Garden,Café,Malay Restaurant,Indian Restaurant,Pizza Place,Supermarket
6227,2310000.0,Pub,Café,Gym / Fitness Center,Indian Restaurant,Persian Restaurant,Garden,Pizza Place,Restaurant,Coffee Shop,Hotel


In [61]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 7, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
196,2450000.0,Café,Indian Restaurant,Pub,Coffee Shop,Restaurant,Bar,Burger Joint,New American Restaurant,Seafood Restaurant,Museum
1553,2456875.0,Food Service,Home Service,Zoo Exhibit,Film Studio,Event Space,Exhibit,Factory,Falafel Restaurant,Farm,Farmers Market
1914,2445000.0,Construction & Landscaping,Bar,Grocery Store,Dance Studio,Athletics & Sports,Film Studio,Exhibit,Factory,Falafel Restaurant,Farm
2136,2461000.0,Pub,Trail,Food,Fast Food Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant,Farm
2606,2450000.0,Park,Garden,Cocktail Bar,Beer Bar,Playground,Gym / Fitness Center,Coffee Shop,Mexican Restaurant,Furniture / Home Store,Performing Arts Venue


In [62]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 8, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
405,2250000.0,Juice Bar,Hotel,Pub,Restaurant,Indian Restaurant,Clothing Store,Italian Restaurant,Café,Supermarket,Mediterranean Restaurant
2638,2250000.0,Cosmetics Shop,Italian Restaurant,Bookstore,Supermarket,Gift Shop,Bakery,Pharmacy,Clothing Store,Coffee Shop,Grocery Store
2808,2250000.0,Pizza Place,Italian Restaurant,Liquor Store,Park,Seafood Restaurant,Performing Arts Venue,Bank,Chinese Restaurant,Sandwich Place,Farmers Market
5241,2258500.0,Business Service,Fish & Chips Shop,Convenience Store,Exhibit,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant
5244,2242500.0,Indian Restaurant,Construction & Landscaping,Photography Studio,Pizza Place,Bus Stop,Food,Flower Shop,Exhibit,Factory,Falafel Restaurant


In [63]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 9, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
422,2400000.0,Pub,Nightclub,Bar,Casino,Food & Drink Shop,Food,Food Service,Exhibit,Factory,Falafel Restaurant
1632,2397132.0,Indian Restaurant,Chinese Restaurant,Health & Beauty Service,Coffee Shop,Grocery Store,Fast Food Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Factory
1797,2400000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2484,2414000.0,Spa,Dry Cleaner,American Restaurant,Photography Studio,Zoo Exhibit,Filipino Restaurant,Exhibit,Factory,Falafel Restaurant,Farm
2641,2410000.0,Café,Pub,Italian Restaurant,Nightclub,Bakery,Juice Bar,Gym / Fitness Center,Furniture / Home Store,Pizza Place,Plaza


## Results and Discussion Section

We may analyze our results according to the clusterd that we produced. Even though, all the clusters could proise an optimal range of facilities and amenities, we have foun certainn patterns.

Clusters 0, 1 and 3 are for the ones who enjoy spendind weekends in clubs and pubs, while cluster 5 and 6 are for theatre lovers.
Investing in clusters 4, 8 and 9 would be the best as they provide the maximum facilities in their neighbourhoods. For the home buyers who are prone to live in 'green' areas should go for areas in cluster 2, 5, 6 and 7 as they have parks, gardens, farms and waterfronts.



## Conclusion

To sum up, in a country with such a huge area and high real estate prices, keeping in mind all the amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores, we aimed at finding an appropriate location to incest in. As a result, we came up with a project, where we recomment the investers a perfect location.

To solve the business problem, we clustered London neighborhoods in order to recommend areas in the required budget. We recommended profitable venues according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores.

First, we gathered data on London properties and the relative price paid data were extracted from the HM Land Registry (http://landregistry.data.gov.uk/). We accessed data through FourSquare API interface and arranged them as a data frame for visualization. By merging data on London properties and the relative price paid data from the HM Land Registry and data on amenities and essential facilities surrounding such properties from FourSquare API interface, we were able to recommend profitable real estate investments.

Second, The Methodology section comprised four stages: 1. Collect Inspection Data; 2. Explore and Understand Data; 3. Data preparation and preprocessing; 4. Modeling. In particular, in the modeling section, we used the k-means clustering technique as it is fast and efficient in terms of computational cost, is highly flexible to account for mutations in real estate market in London and is accurate.

Finally, we concluded recommending the locations to investors according to their interests.